In [1]:
import numpy as np
import os

from src.larp import LARP
from src.get_data import random_data
from src.dows_generator import dows_generator

In [2]:
k_vehicles = 2
Q_vehicle_capacity = 100
facility = 'F'

m_storages = 4
n_fields = 10

use = (n_fields, m_storages)
fields, storages, households, f, q, fs_dist, cs_dist, d = random_data(*use)

In [3]:
larp_model = LARP(facility, 
                k_vehicles, 
                Q_vehicle_capacity, 
                fields, 
                storages, 
                households, 
                f, 
                q, 
                fs_dist, 
                cs_dist, 
                d)

# NOTE: Since it is difficult to find an optimal solution 
# trying with random X, Y and Z (decision variables)
# we fix a random X and set guroby to find and stop the very first feasible solution
larp_model.model.setParam('SolutionLimit', 1)
larp_model.build()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-12
-- LARP model build COMPLETED --


In [4]:
n_dows = 1
generator = dows_generator(larp_model, n_dows, m_storages, n_fields, k_vehicles)
rainfall = list()
for i, dow in enumerate(generator):
    print('i:', i, 'objvalue:', dow.obj_value, sep=' ')
    rainfall.append(dow)

generator loop: 0
dow created
LARP model optimized
LARP status : 10
iter: 0  --> DOW FEASIBLE
i: 0 objvalue: 1.0066711868934668e+18


In [5]:
larp_model.model.resetParams()

In [6]:
dow = rainfall[0]

In [7]:
print(dow.X)

[1 0 1 0]


In [8]:
print(dow.Y)

[[0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]]


In [9]:
print(dow.Z)

[[0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0.]
 [1. 0. 1. 0. 0.]]


In [10]:
# from src.opt_1_neighbourhood import opt_1
# local_optimum = opt_1(larp_model, dow)

In [11]:
from src.swap_neighbourhood import swap
local_optimum = swap(larp_model, dow)

zero_idx: 1 nonzero_idx: 0
model optimization in-progress...
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-5200U CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 272 rows, 109 columns and 724 nonzeros
Model fingerprint: 0x93e0d20d
Variable types: 0 continuous, 109 integer (105 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [2e+00, 2e+18]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.

MIP start from previous solve did not produce a new incumbent solution
MIP start from previous solve violates constraint X_Constr_WaterFlow[0] by 1.000000000

Found heuristic solution: objective 1.071111e+16
Presolve removed 272 rows and 109 columns
Presolve time: 0.00s
Presolve

In [12]:
dow.to_vector()
print(dow)

objValue: 1.0066711868934668e+18
X: [1 0 1 0]
Y: [3 1 1 1 1 3 1 1 1 1]
Z: [0 1 5 0 3 5]



In [13]:
print(local_optimum)

objValue: 1.0711114667433826e+16
X: [0 1 1 0]
Y: [3 2 2 2 2 3 2 2 2 2]
Z: [0 2 5 0 3 5]



In [14]:
dow == local_optimum

False